In [ ]:
model = Sequential()
model.add(Conv2D(64, (2, 2), padding='same', activation='relu', input_shape=(60, 105, 3)))
model.add(Conv2D(64, (2, 2), padding='same', activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(BatchNormalization())
model.add(Dropout(0.25))
model.add(Conv2D(128, (2, 2), padding='same', activation='relu'))
model.add(Conv2D(128, (2, 2), padding='same', activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(BatchNormalization())
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(100, activation='linear'))
model.add(Dense(10, activation='linear'))
model.add(Dense(1, activation='sigmoid'))
#model.load_weights('./drive/MyDrive/model_covid.h5')

data = pd.read_csv('/content/drive/MyDrive/train.csv')
area = (20, 10, 125, 70)
false_samples = [] # примеры ложных срабатываний
true_samples = [] # примеры верных срабатываний
pred_true = 0 
pred_false = 0
true_value = data['y'].values # значения из столбца y
samples_pred = np.zeros(shape=(len(true_value),))
all_count = data['x'].values # значения из столбца x
number_segments = 0 # считаем сколько всего отрезков
for i in range(5,len(all_count)-3):
  number_segments += 1
  if all_count[i-2] < all_count[i-1] > all_count[i] < all_count[i+1]:
    t2 = all_count[i-1]-all_count[i]
    if 40 <= t2 <= 165 and all_count[i-1] >= 560 and all_count[i] >= 464:  # 40 <= t2 <= 165         78 <= t2 <= 130
      if abs((all_count[i+1]-all_count[i])+(all_count[i-1]-all_count[i])+(all_count[i-1]-all_count[i-2])) <= 320:
        fig, ax = plt.subplots()
        ax.plot([all_count[i-2], all_count[i-1], all_count[i], all_count[i+1]])
        fig.savefig('/content/for_verification')
        for_cr = image.load_img('/content/for_verification.png', target_size=(80,140))
        cropped_img = for_cr.crop(area)
        img = np.asarray(cropped_img).reshape(1,60,105,3)
        img = img/255.
        if (np.round(model.predict(img)[0],4)) >= 0.2: # порог пропуска ковидных значений
          #samples_pred[i-7] = 1
          #samples_pred[i-6] = 1
          samples_pred[i-5] = 1
          samples_pred[i-4] = 1
          samples_pred[i-3] = 1
          samples_pred[i-2] = 1
          samples_pred[i-1] = 1
          samples_pred[i] = 1
          samples_pred[i+1] = 1
          samples_pred[i+2] = 1
          samples_pred[i+3] = 1
          #samples_pred[i+4] = 1
          #samples_pred[i+5] = 1
          #samples_pred[i+6] = 1
          if true_value[i-2] == true_value[i-1] == true_value[i] == true_value[i+1] == 1:
            pred_true += 1
            true_samples.append([all_count[i-2], all_count[i-1], all_count[i], all_count[i+1]])

          else:
            pred_false += 1
            false_samples.append([all_count[i-2], all_count[i-1], all_count[i], all_count[i+1]])
print('Всего отрезков:', number_segments)
print()
print('Из них:')
print('Алгоритм распознал как ковидные:', pred_true + pred_false)
print('Количество верных ответов:', pred_true)
print('Количество не верных ответов:', pred_false)
print()
print('Итого:')
print('Распознано ковидных значений:', pred_true/1000*100, '%') # 1000 ковидных аномалий
print('Распознано нормальных значений:', 100 - (pred_false / (number_segments - 1000) * 100), '%')
print()
f1 = f1_score(samples_pred, true_value, average='binary')
f1